In [1]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0)


In [3]:
#walking, running, jumping키워드가 있는 데이터만 추출

import pandas as pd

# CSV 파일 읽기
file_path = 'mpii_human_pose.csv'  # CSV 파일 경로를 지정하세요.
df = pd.read_csv(file_path)

# 특정 단어를 포함한 행만 필터링
keywords = ['walking', 'running', 'jumping']
pattern = '|'.join(keywords)  # 'walking|running|jumping' 형태의 패턴 생성

filtered_df = df[df['Activity'].str.contains(pattern, case=False, na=False)]

# 결과를 새로운 CSV 파일로 저장
output_file_path = 'filtered_file.csv'  # 결과를 저장할 파일 경로를 지정하세요.
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered data saved to {output_file_path}")


Filtered data saved to filtered_file2.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 예시: 특징 벡터와 라벨을 준비합니다.
X = [...]  # 특징 벡터
y = [...]  # 라벨 (0: 걷기, 1: 뛰기, 2: 점프, ...)

# 데이터를 학습과 테스트 세트로 나눕니다.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 분류 모델을 학습합니다.
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 모델을 평가합니다.
accuracy = model.score(X_test, y_test)
print(f"Accuracy: {accuracy:.2f}")


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    
    if results.pose_landmarks:
        # 관절 위치를 기반으로 특징을 추출합니다.
        features = extract_features(results.pose_landmarks)
        
        # 모델을 사용하여 동작을 예측합니다.
        prediction = model.predict([features])
        
        # 예측 결과를 화면에 표시합니다.
        action = ['걷기', '뛰기', '점프'][prediction[0]]
        cv2.putText(frame, action, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    cv2.imshow('Mediapipe Pose', frame)
    
    if cv2.waitKey(5) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [4]:
import pandas as pd

# CSV 파일 읽기
file_path = 'filtered_file.csv'  # CSV 파일 경로를 지정하세요.
df = pd.read_csv(file_path)

# 특정 단어를 포함한 행만 필터링하고 해당 단어로 바꾸기
keywords = ['walking', 'running', 'jumping']
pattern = '|'.join(keywords)  # 'walking|running|jumping' 형태의 패턴 생성

def extract_keyword(activity):
    for keyword in keywords:
        if keyword in activity.lower():
            return keyword
    return None

df['Activity'] = df['Activity'].apply(extract_keyword)

# 'Activity' 열이 NaN이 아닌 행만 남기기
filtered_df = df.dropna(subset=['Activity'])

# 결과를 새로운 CSV 파일로 저장
output_file_path = 'filtered_file2.csv'  # 결과를 저장할 파일 경로를 지정하세요.
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered data saved to {output_file_path}")


Filtered data saved to filtered_file2.csv


In [11]:
import pandas as pd

# 업로드된 파일 경로
file_path = 'filtered_file2.csv'

# 파일을 읽어서 데이터프레임으로 변환
try:
    # 먼저 탭으로 구분된 파일로 시도
    df = pd.read_csv(file_path, delimiter='\t')
except:
    # 탭 구분자가 아니면 콤마 구분자로 시도
    df = pd.read_csv(file_path, delimiter=',')

# 데이터프레임 출력
print(df.head())

# ID와 NAME, Category 열은 분석에 사용되지 않으므로 제거
features = df.drop(columns=['ID', 'NAME', 'Category'])
labels = df['Activity']

# 출력 및 확인
print("Features:")
print(features.head())
print("\nLabels:")
print(labels.head())

# 특징 벡터와 라벨을 파일로 저장할 경우 (예: CSV 파일)
features.to_csv('/mnt/data/features.csv', index=False)
labels.to_csv('/mnt/data/labels.csv', index=False)

print("Features and labels have been saved.")


  ID,NAME,r ankle_X,r ankle_Y,r knee_X,r knee_Y,r hip_X,r hip_Y,l hip_X,l hip_Y,l knee_X,l knee_Y,l ankle_X,l ankle_Y,pelvis_X,pelvis_Y,thorax_X,thorax_Y,upper neck_X,upper neck_Y,head top_X,head top_Y,r wrist_X,r wrist_Y,r elbow_X,r elbow_Y,r shoulder_X,r shoulder_Y,l shoulder_X,l shoulder_Y,l elbow_X,l elbow_Y,l wrist_X,l wrist_Y,Scale,Activity,Category
0  14,029122914.jpg,-1,-1,301,473,302,346,362,345...                                                                                                                                                                                                                                                                                                                 
1  15,061185289.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1...                                                                                                                                                                                                                                        

KeyError: "['ID', 'NAME', 'Category'] not found in axis"